# NNのデモコード

### いくつかエラーが出るようにしているので、調べながら修正してみてください。

### データセットをモデルに入れるための準備

必要なもののインポート

In [14]:
import torch
import numpy as np
import torch.utils.data as data
import pandas as pd

### データリスト関数

In [15]:
def makeDataList(csv_path):
    datalist = pd.read_csv(csv_path)
    datalist = datalist.drop(["Name", "Ticket", "Cabin"], axis=1)
    datalist = pd.get_dummies(datalist)
    datalist = datalist.fillna(-1)
    return datalist
datalist = makeDataList("/workspace/dataset/train.csv")

確認

In [ ]:
print("datalist.values[0] =", datalist.values[0])
datalist

### データ分割

In [17]:
from sklearn.model_selection import train_test_split

train_datalist, val_datalist = train_test_split(datalist, test_size=0.1, random_state=1234, shuffle=True)

確認

In [ ]:
print("len(train_datalist) =", len(train_datalist))
print("len(val_datalist) =", len(val_datalist))

### データリスト関数

In [19]:
class DatasetMaker(data.Dataset):
    def __init__(self, datalist):
        self.input_datalist = datalist.drop(["PassengerId", "Survived"], axis=1).values.astype(np.float32)
        self.label_datalist = datalist["Survived"].values.astype(np.int64)

    def __len__(self):
        return len(self.input_datalist)

    def __getitem__(self, index):
        inputs = self.input_datalist[index]
        labels = self.label_datalist[index]
        return inputs, labels

dataset = DatasetMaker(datalist)

確認

In [ ]:
print("dataset.__len__() =", dataset.__len__())
print("dataset.__getitem__(index=0)[0] =", dataset.__getitem__(index=0)[0])
print("dataset.__getitem__(index=0)[1] =", dataset.__getitem__(index=0)[1])

### データローダーの確認
 torch.utils.data.DataLoader()を呼び出すと使える

In [ ]:
batch_size = 5
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
batch_itr = iter(dataloader)
inputs, labels = next(batch_itr)

print("inputs =\n", inputs)
print("inputs.size() =", inputs.size())
print("labels =", labels)
print("labels.size() =", labels.size())

### ネットワーククラスの定義

In [22]:
from torch import nn

class Network(nn.Module):
    def __init__(self, dim_inputs, dim_mid, dim_outputs, dropout_rate):
        super().__init__()

        self.fc = nn.Sequential(
            nn.Linear(dim_inputs, dim_mid),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            
            nn.Linear(dim_mid, 32),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            
            nn.Linear(dim_mid, dim_outputs)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

net = Network(len(dataset.__getitem__(index=0)[0]), 64, 2, dropout_rate=0.1)

確認

In [ ]:
print(net)
outputs = net(inputs)
print("outputs = \n", outputs)
print("outputs.size() =", outputs.size())

### Trainerクラスを定義

In [24]:
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.optim as optim

class Trainer:
    def __init__(self, csv_path, num_epochs, batch_size, lr, save_weights_path):
        # デバイスの設定（GPUが利用可能な場合はGPU、そうでない場合はCPUを使用）
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        print("self.device =", self.device)

        # トレーニングのパラメータを設定
        self.num_epochs = num_epochs
        self.save_weights_path = save_weights_path

        # データの準備
        datalist = makeDataList(csv_path)  # CSVファイルからデータリストを作成
        # データを訓練用と検証用に分割（90%訓練、10%検証）
        train_datalist, val_datalist = train_test_split(datalist, test_size=0.1, random_state=1234, shuffle=True)
        
        # データセットとデータローダーの作成
        train_dataset = DatasetMaker(train_datalist)
        val_dataset = DatasetMaker(val_datalist)
        train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
        val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
        
        # データローダーを辞書にまとめる
        self.dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

        # ネットワークモデルの初期化
        self.net = Network(
            dim_inputs = len(train_dataset.__getitem__(index=0)[0]),
            dim_mid = 64,
            dim_outputs = 2,
            dropout_rate=0.1
        )
        self.net.to(self.device)  # モデルをGPUまたはCPUに移動

        # 損失関数と最適化アルゴリズムの設定
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.net.parameters(), lr=lr)


    def train(self):

        # 損失の記録用辞書
        record_loss_dict = {"train": [], "val": []}
        min_loss_epoch = 0.0

        # エポックのループ
        for epoch in range(self.num_epochs):
            if epoch == 0 or not (epoch+1) % (self.num_epochs // 10):
                print("----------")
                print("Epoch {}/{}".format(epoch+1, self.num_epochs))

            # 訓練フェーズと検証フェーズのループ
            for phase in ["train", "val"]:
                # モデルのモード設定（訓練時はtrain、検証時はeval）
                if phase == "train":
                    self.net.train()
                else:
                    self.net.eval()

                # 損失と入力数の初期化
                loss_epoch = 0.0
                num_inputs = 0

                # ミニバッチのループ
                for inputs, labels in self.dataloaders_dict[phase]:
                    inputs = inputs.to(self.device)
                    labels = labels.to(self.device)

                    # 勾配のリセット
                    self.optimizer.zero_grad()

                    # 訓練時のみ勾配を計算
                    with torch.set_grad_enabled(phase == "train"):
                        # 順伝播
                        outputs = self.net(inputs)
                        loss = self.criterion(outputs, labels)

                        # 訓練時は逆伝播と最適化
                        if phase == "train":
                            loss.backward()
                            self.optimizer.step()

                    # 損失の累積
                    loss_epoch += loss.item() * inputs.size(0)
                    num_inputs += inputs.size(0)

                # エポックごとの平均損失を計算
                loss_epoch = loss_epoch / num_inputs
                record_loss_dict[phase].append(loss_epoch)

                if epoch == 0 or not (epoch+1) % (self.num_epochs // 10):
                    print("{} Loss: {:.4f}".format(phase, loss_epoch))

            # 最良モデルの保存
            if epoch == 0 or record_loss_dict["val"][-1] < min_loss_epoch:
                min_loss_epoch = record_loss_dict["val"][-1]
                torch.save(self.net.state_dict(), self.save_weights_path)

        # 損失のグラフ表示
        self.showGraph(record_loss_dict)

    def showGraph(self, record_loss_dict):
        # 訓練と検証の損失をグラフ化
        graph = plt.figure()
        plt.plot(range(len(record_loss_dict["train"])), record_loss_dict["train"], label="Training")
        plt.plot(range(len(record_loss_dict["val"])), record_loss_dict["val"], label="Validation")
        plt.legend()
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.title("last loss: train=" + str(record_loss_dict["train"][-1]) + ", val=" + str(record_loss_dict["val"][-1]))
        plt.show()

## モデルの訓練

In [25]:
if __name__ == '__main__':
    csv_path = "workspace/dataset/train.csv"
    num_epochs = 2000
    batch_size = 80
    lr = 0.0001
    save_weights_path = "./weights.pth"

    trainer = Trainer(csv_path, num_epochs, batch_size, lr, save_weights_path)
    trainer.train()

----------
Epoch 200/2000
train Loss: 0.5186
val Loss: 0.4571
----------
Epoch 400/2000
train Loss: 0.4760
val Loss: 0.4256


KeyboardInterrupt: 

# 課題内容

## 1. コードの分離と再構築
- `demo.ipynb`の内容を以下のように機能別に分離してください:
  - `code/model.py`
  - `code/train.py`
  - `code/utils.py`
- `train.py`を実行すると動作するように実装してください
- 確認用のコードは不要です

## 2. 推論の実装
- 新しいファイル`code/predict.py`を作成してください
- `dataset/test.csv`を入力データとして使用してください
- 学習した重みをモデルで読み込んで推論を実行してください
  - 注意: test.csvには`Survived`ラベルは含まれていないです
- 推論結果を`result`フォルダ内にCSVファイルとして保存してください


## 課題
 #### ・demo.ipynbの中身を機能ごとにcodeフォルダの中のmodel.py,train.py,utils.pyに分離して、train.pyを実行すると動くようにしてみましょう
  #### *確認部分はなくていいです
 #### ・datasetの中のtest.csvを入力データとして実際の推論を行ってみましょう 
　
  ##### test.csvにはSurvivedのラベルは無いので、読み込んで学習したモデルに入力し、出力をcsvファイルとしてresultフォルダに保存してください